In [2]:
import tensorflow as tf
import numpy as np
from predictor import Predictor
from models import *
import matplotlib.pyplot as plt
import pandas as pd
import math
from tensorflow.keras.layers import *

In [4]:
sections = 6
n = sections + 1
inputs = Input(shape=(2**sections,), name="inputs")
# x = inputs
# l1 = Lambda(lambda z: z[:, -(2 ** sections) :])
# l2 = [Lambda(lambda z: z[0][:, -(2 ** z[1]) :]) for i in range(n)]
a = tf.constant(np.arange(16, dtype='float32').reshape((4,4)))
y = tf.nn.moments(a, axes=[1], keepdims=True)
z = tf.math.subtract(a,y[0])
w = tf.math.divide(z,y[1])
# print(a.numpy())
print(w)
# a1=l1(a)
# a2=[l2[i]((a1,i)) for i in range(n)]
# a2


tf.Tensor(
[[-1.2 -0.4  0.4  1.2]
 [-1.2 -0.4  0.4  1.2]
 [-1.2 -0.4  0.4  1.2]
 [-1.2 -0.4  0.4  1.2]], shape=(4, 4), dtype=float32)


In [ ]:
x = [Conv1D(16, 2 ** i, padding="valid", name=f"c1-{i}")(x[i]) for i in range(n)]
x = [Lambda(lambda z: z[:, -(2 ** i) :, :])(x) for i in range(n)]
x = [Flatten()(x[i]) for i in range(n)]
x = Concatenate(axis=1)(x)
x = Flatten()(x)
x = Dense(units, "sigmoid")(x)
x = Dense(64, "tanh")(x)
x = Dense(8, "tanh")(x)
x = Dense(input_shape[-1], kernel_regularizer=l2)(x)
outputs = x
model = keras.Model(inputs, outputs, name="trendencoder")
MAE = keras.metrics.MeanAbsoluteError()
RMSE = keras.metrics.RootMeanSquaredError()
model.compile(
    loss=keras.losses.MeanSquaredError(),
    optimizer=keras.optimizers.Adam(learning_rate=1e-2),
    metrics=[MAE],
)
print(model.summary())


In [6]:
y_true = [[-1., 1.], [1., -1.]]
y_pred = [[1., -1.], [-1., 1.]]
# Using 'auto'/'sum_over_batch_size' reduction type.

# mse = tf.keras.losses.MeanSquaredError(reduction=tf.keras.losses.Reduction.SUM)
mse = tf.keras.losses.MeanSquaredError()
mse(y_true, y_pred).numpy()


1.0